# Vellend's (Moran) Drift Model
## python edition

### Getting started
First we have to import a bunch of libraries to do the things we want to do. I will be using:
- numpy: let's us do vectorized math and has lots of useful functions (I do not endore the numpy "core team" they are assholes)
- matplotlib: let's us make plots (not as good as ggplot, seem like good people)
- seaborn: more plotting options (also not as good as ggplot)
- tqdm: progress bar, these models get slooooow

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from tqdm.notebook import tqdm
%matplotlib inline
%matplotlib widget

## The Model itself
I'm going to implement the basic Moran model shown in Ch. 6. My version will be contained as a function so we can interactively change parameter values without having to actually edit the code (yuck)

In [5]:
def moran_model(n_indiv, n_years, init_f1):
    """ Implements the Moran model and simulates it in time. 
        
        Parameters
        ----------
        n_indiv: int
            number of individuals in the community
        n_years: int
            number of potential 'turnovers' of the community
        init_f1: float
            initial frequency of species 1
            
        Return
        ------
        moran: np.array (n_years, 2)
            contains the species frequencies for each year of the simulation """

    # set up an empty array for the simulated frequencies
    # initialize with the given frequency. python counts from 0, sorrryyyy
    moran = np.zeros((n_years, 2))
    moran[0] = np.array([init_f1, 1 - init_f1])

    # get a vector representing the community. vellend calls this COM
    # this just makes a random vector drawn from the initial frequency
    comm = np.random.choice([0, 1], size=n_indiv, replace=True, p=moran[0])

    # now we can do the loop as in vellend. I'm going to write it a bit differently
    # but the idea is basically the same. (yes its slower)
    for year in range(1, n_years):

        # for each year we potentially replace each individual so we can loop
        # over that as well
        for indiv in range(n_indiv):
            # get probability of species 1
            f1 = np.sum(comm == 0) / n_indiv

            # replace one individual with another
            death = np.random.randint(n_indiv)
            birth = np.random.choice([0, 1], p=[f1, 1 - f1])
            comm[death] = birth
        
        # when we're done looping over all of the individuals we can update
        # frequencies for the year (the timescale we care about)
        f1 = np.sum(comm == 0) / n_indiv
        moran[year] = np.array([f1, 1 - f1])

    return moran

### Running the simulations and visualization
now we can actually run this thing. One run isn't very interesting so we're going to run a bunch. We can change the parameters for the model in the little block at the top of this next cell

In [7]:
def draw_simulation(iterations, individuals, years, initial_freq):
    # the plot bit, this just makes a blank plot
    fig, ax = plt.subplots(ncols=2, figsize=(10,4), sharey=True, gridspec_kw = {'width_ratios':[3, 1]})
    # trajectory labels
    ax[0].set_xlabel('Years')
    ax[0].set_ylabel('Species 1 frequency')
    ax[0].set_ylim([0, 1])
    # distribution labels
    ax[1].set_xlabel('Count')
    ax[1].set_xlim([0, iterations])

    # we're going to plot a distribution too need an array for it
    dist = np.zeros(iterations)

    # we run the simulation and draw a trajectory
    for i in tqdm(range(iterations)):
        simulation = moran_model(individuals, years, initial_freq)
        ax[0].plot(range(years), simulation[:, 0])
        # add final freq to our dist array
        dist[i] = simulation[-1, 0]

    # plot the distribution too
    ax[1].hist(dist, orientation='horizontal')
    ax[1].axhline(np.mean(dist), linestyle='--')


    plt.tight_layout()    
    plt.show()

# set up the interface
widgets.interact_manual(draw_simulation, iterations=(5, 50, 5), individuals=(10, 500, 10), years=(5, 200, 5),
                  initial_freq=(0.0, 1.0, 0.01))

interactive(children=(IntSlider(value=25, description='iterations', max=50, min=5, step=5), IntSlider(value=50…

<function __main__.draw_simulation(iterations, individuals, years, initial_freq)>